In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import skimage
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Load images with numpy
import csv

        
        
images = np.load('../input/f2018-hand-drawn-pictures/train_images.npy', encoding="latin1")
test_images = np.load('../input/f2018-hand-drawn-pictures/test_images.npy', encoding="latin1")
labels=[]
with open('../input/f2018-hand-drawn-pictures/train_labels.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for ln in reader:
        labels.append(ln)
#create a y array target
labels = np.array(labels[1:])
target = labels[:,1]
print(target)
print(test_images.shape)
target_extend = np.repeat(target, 4)
print(len(target_extend))

In [ ]:
import skimage
from skimage.restoration import estimate_sigma
from skimage.transform import rotate
import cv2 as cv
from scipy import ndimage
#Reshaping image to 100x100
image_list_extend = []
image_list_test = []
image_list = []
#doing data augmentation:
for im in images:
    temp = im[1].reshape(100,100)
    temp2 = np.fliplr(temp) #flipping left-right
    temp3 = np.flipud(temp) #flipping up/down
    temp4 = rotate(temp, angle=45, mode='reflect') 
    
    image_list_extend.append(temp)
    image_list_extend.append(temp2)
    image_list_extend.append(temp3)
    image_list_extend.append(temp4)    
    
    image_list.append(temp)
print(len(image_list))
print(len(image_list_extend))
    
#reshaping:
for im in test_images:
    temp = im[1].reshape(100,100)
    temp2 = temp
    image_list_test.append(temp)
    


In [ ]:
#a function that proccesses the image:
import cv2
import math
import scipy.stats
import os.path
from tqdm import tqdm


def process(img_list):
    img_ps = []
    for i in tqdm(range(img_list.shape[0]), desc='image'):
        
        # convert to binary images
        #im = img_list[i] / 255.
        im = img_list[i]
        im = im.astype(np.uint8)

        # thresholding
        ret,thresh = cv2.threshold(im,127,255,0)
        
        # find the contours from the image
        im2, im_contours, h = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        
        c_radii = []
        
        # find the minimum enclosing circle of the contour, add the radius to list of radii
        for contour in im_contours:
            _cc, radius = cv2.minEnclosingCircle(contour)
            c_radii.append(radius)

        # find maximum radii from list, which is the largest contour in the image
        sort_radii = sorted(c_radii, reverse=True)
        largest = c_radii.index(sort_radii[0])

        # store only the largest contour from the image
        cx, cy, cw, ch = cv2.boundingRect(im_contours[largest])
        im2 = im[cy:cy + ch, cx:cx + cw]

        # resize the image to a 32x32 array
        im3 = cv2.resize(im2, (32, 32))
        img_ps.append(im3)

    return img_ps

In [ ]:
#apply the processsing to out train and test
np.set_printoptions(threshold=np.nan)
training_ps = process(np.array(image_list))
train_ps_extend = process(np.array(image_list_extend))
test_ps = process(np.array(image_list_test))
X = np.array(training_ps)
X_extend = np.array(train_ps_extend)
X_test_im = np.array(test_ps)
print(X.shape)
print(X_test_im.shape)
plt.imshow(test_ps[11])
#plt.imshow(image_list_test[11])
#print(training_ps[1])

In [ ]:
#SVM baseline is here:

In [ ]:
#splitting and flattening for SVM
from sklearn.model_selection import train_test_split
new_images_train = []
new_images_test = []
for image in training_ps:
    image = image.flatten()
    new_images_train.append(image)
new_images_train  = np.array(new_images_train)    

# for image in test_ps:
#     image = image.flatten()
#     new_images_test.append(image)
# new_images_test = np.array(new_images_test) 
# print(new_images_test.shape)
# print(new_images_train.shape)

X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(new_images_train, target)
X_train_svm = np.array(X_train_svm)
X_test_svm = np.array(X_test_svm)

y_train_svm = np.array(y_train_svm)
y_test_svm = np.array(y_test_svm)
print(X_train_svm.shape)
print(X_test_svm.shape)
print(y_train_svm.shape)
print(y_test_svm.shape)

In [ ]:
#SVM
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

clf = LinearSVC(verbose=True, penalty='l2', loss='hinge', C=0.000001, tol=0.0001)
clf.fit(X_train_svm, y_train_svm)
SVM_prediction = clf.predict(X_test_svm)
print("SVC Accuracy :", accuracy_score(y_test_svm, SVM_prediction))


In [ ]:
#splitting for CNN
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training_ps, target)
X_train_extend, X_test_extend, y_train_extend, y_test_extend = train_test_split(train_ps_extend, target_extend)
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train_extend = np.array(X_train_extend)
X_test_extend = np.array(X_test_extend)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
#Convolutional Neural network

In [ ]:
from __future__ import print_function
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

batch_size = 128
num_classes = 31
epochs = 100

# input image dimensions
img_rows, img_cols = 32, 32
X_train_extend = X_train_extend.reshape(X_train_extend.shape[0], 1, img_rows, img_cols)
X_test_extend = X_test_extend.reshape(X_test_extend.shape[0], 1, img_rows, img_cols)
X_test_im = X_test_im.reshape(X_test_im.shape[0], 1, img_rows, img_cols)
input_shape = (1, img_rows, img_cols)

lbl_encode = LabelEncoder()
y_lab_train = lbl_encode.fit_transform(y_train_extend)
y_lab_test = lbl_encode.transform(y_test_extend)

y_lab_train = y_lab_train.reshape(len(y_lab_train), 1)
y_lab_test = y_lab_test.reshape(len(y_lab_test), 1)

encoder = OneHotEncoder(sparse=False)
y_encode_train = encoder.fit_transform(y_lab_train)
y_encode_test = encoder.transform(y_lab_test)


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=input_shape, data_format = 'channels_first', activation = 'relu'))
model.add(Conv2D(64, (3, 3), data_format = 'channels_first', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), padding='same', data_format = 'channels_first', activation = 'sigmoid'))
model.add(Conv2D(64, (3, 3), activation = 'sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='Adam',metrics=['accuracy'])
earlyStopping = EarlyStopping(monitor='val_acc', patience=10, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model-best.h5', verbose=1, monitor='val_acc',save_best_only='true', mode='auto') 
history = model.fit(X_train_extend, y_encode_train, batch_size=batch_size, epochs=epochs, verbose=1,callbacks=[earlyStopping, checkpoint], validation_data=(X_test_extend, y_encode_test))
model.load_weights(filepath = 'model-best.h5')
result = model.evaluate(X_test_extend, y_encode_test, verbose=0)
print('Test loss:', result[0])
print('Test accuracy:', result[1])


#bin_pred = model.predict_classes(X_test_im)
# # # # # # # #print(bin_pred)
#fin_pred = lbl_encode.inverse_transform(bin_pred)

#CNN_predict = pd.DataFrame(columns=["Id","Category"])
#CNN_predict["Id"] = range(0,10000) 
# CNN_predict["Category"] =  fin_pred
# CNN_predict.to_csv("CNN7_submission.csv", index=False)

In [ ]:


from __future__ import print_function
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K

batch_size = 128
num_classes = 31
epochs = 50

# input image dimensions
img_rows, img_cols = 32, 32
X_train_extend = X_train_extend.reshape(X_train_extend.shape[0], 1, img_rows, img_cols)
X_test_extend = X_test_extend.reshape(X_test_extend.shape[0], 1, img_rows, img_cols)
X_test_im = X_test_im.reshape(X_test_im.shape[0], 1, img_rows, img_cols)
input_shape = (1, img_rows, img_cols)

lbl_encode = LabelEncoder()
y_lab_train = lbl_encode.fit_transform(y_train_extend)
y_lab_test = lbl_encode.transform(y_test_extend)

y_lab_train = y_lab_train.reshape(len(y_lab_train), 1)
y_lab_test = y_lab_test.reshape(len(y_lab_test), 1)

encoder = OneHotEncoder(sparse=False)
y_encode_train = encoder.fit_transform(y_lab_train)
y_encode_test = encoder.transform(y_lab_test)
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', 
                input_shape=input_shape, data_format = 'channels_first'))
model.add(Conv2D(64, (3, 3), activation='sigmoid', data_format = 'channels_first'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
earlyStopping = EarlyStopping(monitor='val_acc', patience=10, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5', verbose=1, monitor='val_acc',save_best_only='true', mode='auto') 
# model.fit(X_train_extend, y_encode_train, batch_size=batch_size, epochs=epochs, verbose=1,callbacks=[earlyStopping, checkpoint], validation_data=(X_test_extend, y_encode_test))
# #model.load_weights(filepath = 'model-{epoch:03d}.h5')
# score = model.evaluate(X_test_extend, y_encode_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

# bin_pred = model.predict_classes(X_test_im)
# # #print(bin_pred)
# fin_pred = lbl_encode.inverse_transform(bin_pred)

# CNN_predict = pd.DataFrame(columns=["Id","Category"])
# CNN_predict["Id"] = range(0,10000) 
# CNN_predict["Category"] =  fin_pred
# CNN_predict.to_csv("CNN3_submission.csv", index=False)

print("done-ish")